In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sn
from sklearn.feature_extraction.text import TfidfVectorizer 
from IPython.display import display
# from imblearn.over_sampling import SMOTE, RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score
from sklearn.linear_model import LogisticRegression
from skmultilearn.problem_transform import LabelPowerset
import multiprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from collections import Counter
%matplotlib inline

In [2]:
test_data = pd.read_csv("test.csv")
test_labels = pd.read_csv("test_labels.csv")
train_data = pd.read_csv("train.csv")
test_merged = pd.merge(test_data, test_labels, on='id')

test_merged = test_merged.drop(test_merged[test_merged.toxic == -1].index)

In [3]:
cores = multiprocessing.cpu_count()

y_train = train_data.iloc[:, 2:]
X_train = train_data[['comment_text']]

y_test = test_merged.iloc[:, 2:]
X_test = test_merged[['comment_text']]

# lp = LabelPowerset()
# y_train = lp.transform(y_train)
# y_test = lp.transform(y_test)

tvec = TfidfVectorizer(min_df=.0025, stop_words='english', strip_accents='unicode', analyzer='word', max_features=1000)
X_train = tvec.fit_transform(X_train['comment_text'])
X_test = tvec.transform(X_test['comment_text'])

In [4]:
y_test

,toxic,severe_toxic,obscene,threat,insult,identity_hate
5,0,0,0,0,0,0
7,0,0,0,0,0,0
11,0,0,0,0,0,0
13,0,0,0,0,0,0
14,0,0,0,0,0,0
...,...,...,...,...,...,...
153150,0,0,0,0,0,0
153151,0,0,0,0,0,0
153154,0,0,0,0,0,0
153155,1,0,1,0,1,0


In [5]:
y_train

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
...,...,...,...,...,...,...
159566,0,0,0,0,0,0
159567,0,0,0,0,0,0
159568,0,0,0,0,0,0
159569,0,0,0,0,0,0


In [6]:
y_test_as_np = y_test.to_numpy()

In [7]:
features = y_train.columns

In [8]:
for i, f in enumerate(features):
    print(i, f)

0 toxic
1 severe_toxic
2 obscene
3 threat
4 insult
5 identity_hate


In [9]:
def change_y_to_0_stage_version(y):
    return [1 if any(row.values) else 0 for i, row in y.iterrows()]

In [10]:
class TwoStageClassifier:
    def __init__(self, X_train, X_test, y_train, clf0=None, clfs=None):
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        
        if clf0 is None:
            clf0 = SVC()
        if clfs is None:
            clfs = [SVC() for i in range(self.y_train.shape[1])]
            
        self.clf0 = clf0
        self.clfs = clfs
        
        self.y_pred_0_stage = None
        self.y_pred_1_stage = None
        
    def set_clf0(self, clf0):
        self.clf0 = clf0
        self._clear_y_pred()
        
    def set_clfs(self, clfs):
        self.clfs = clfs
        self._clear_y_pred()

        
    def predict(self):
        return self.predict_1_stage()
    
    def predict_1_stage(self):
        if self.y_pred_1_stage is None:
            self._proceed_1_stage()
        return self.y_pred_1_stage
        
    def predict_0_stage(self):
        if self.y_pred_0_stage is None:
            self._proceed_0_stage()
        return self.y_pred_0_stage
    
    def _proceed_1_stage(self):
        if self.y_pred_0_stage is None:
            self._proceed_0_stage()
        
        features_num = self.y_train.shape[1]
        y_pred_as_np = np.zeros((self.X_test.shape[0], features_num))
        y_train_1_stages = [[] for i in range(features_num)]
        inds = []
        for i, row in self.y_train.iterrows():
            if not any(row.values):
                continue
            inds.append(i)
            for j, v in enumerate(row.values):
                y_train_1_stages[j].append(v)
                
        y_train_1_stages = np.array(y_train_1_stages)
        X_train_1_stage = self.X_train[inds]
        
        for (i, clf) in enumerate(self.clfs):
            clf.fit(X_train_1_stage, y_train_1_stages[i])
        
        y_pred_1_stages = [[] for i in range(features_num)]
        for (i, clf) in enumerate(self.clfs):
            y_pred_1_stages[i] = clf.predict(self.X_test)    # Do predictions for each class
        y_pred_1_stages = np.array(y_pred_1_stages)
        
        for (i, y0) in enumerate(self.y_pred_0_stage):
            if y0 == 0:
                y_pred_as_np[i,:] = 0                       # In case of first stage determining no label we write no labels there
            else:
                y_pred_as_np[i,:] = y_pred_1_stages[:,i]    # In case of first stage determining at least label we write labels from second stage
        
        self.y_pred_1_stage = y_pred_as_np
        
    def _proceed_0_stage(self):
        y_train_0_stage = change_y_to_0_stage_version(self.y_train)
        self.clf0.fit(self.X_train, y_train_0_stage)
        self.y_pred_0_stage = self.clf0.predict(self.X_test)
        
    def _clear_y_pred(self):
        self.y_pred_0_stage = None
        self.y_pred_1_stage = None

In [11]:
def print_predicted_values_statistics(y_test, y_pred, title="Classification results:", algorithm=None):
    print(title)
    if algorithm is not None:
        print("Used algorithm:", algorithm.__class__.__name__)
    acc_score = accuracy_score(y_test, y_pred)
    print("Accuracy score: " + str(acc_score))
    f1_score_value = f1_score(y_test, y_pred, average = 'weighted')
    print("F1 score: " + str(f1_score))
    rec_score = recall_score(y_test, y_pred, average = 'weighted')
    print("Recall score: " + str(rec_score))
    print()
    return {"accuracy_score": acc_score, "f1_score": f1_score_value, "recall_score": rec_score}

In [12]:
# Returns 1.0 - Hamming-Loss measure - number of labels correctly classified
def percent_of_labels_correctly_predicted(y_test, y_pred):
    return np.sum(y_test == y_pred) / y_test.size

In [13]:
def test_two_stage_classifier_with_algorithms(clf0=None, clfs=None):
    my_classifier = TwoStageClassifier(X_train=X_train, X_test=X_test, y_train=y_train)
    if clf0 is not None:
        my_classifier.set_clf0(clf0)
    if clfs is not None:
        my_classifier.set_clfs(clfs)
    
    scores_dict = {}
    y_pred_0_stage = my_classifier.predict_0_stage()
    y_test_0_stage = change_y_to_0_stage_version(y_test)
    first_stage_scores = print_predicted_values_statistics(y_test=y_test_0_stage, y_pred=y_pred_0_stage, 
                                                           title="Test data - first stage classification", 
                                                           algorithm=my_classifier.clf0)
    print
    scores_dict['first_stage'] = first_stage_scores
    
    y_pred_as_np = my_classifier.predict()
    for (i, f) in enumerate(features):
        y_test_f = y_test_as_np[:,i]
        y_pred_f = y_pred_as_np[:,i]
        category_scores = print_predicted_values_statistics(y_test=y_test_f, y_pred=y_pred_f, 
                                                            title="Tested category: "+str(f), 
                                                            algorithm=my_classifier.clfs[i])
        scores_dict[str(f)] = category_scores
    
    overall_acc_score = accuracy_score(y_test_as_np, y_pred_as_np)
    print("Overall accuracy (subset) score (exact match ratio):", 
          overall_acc_score)
    scores_dict['overall_accuracy_score'] = overall_acc_score
    total_percent_of_correct_labels = percent_of_labels_correctly_predicted(y_test_as_np, y_pred_as_np)
    print("Total percent of labels correctly predicted:", 
          total_percent_of_correct_labels)
    scores_dict['total_correct_labels'] = total_percent_of_correct_labels
    return my_classifier, scores_dict

In [14]:
random_state = 44

In [15]:
svc_clf0 = SVC(random_state=random_state)
svc_clfs = [SVC(random_state=random_state) for f in features]

In [16]:
svc_for_both_stages_classifier, svc_for_both_stages_results = \
    test_two_stage_classifier_with_algorithms(clf0=svc_clf0, clfs=svc_clfs)

Test data - first stage classification
Used algorithm: SVC
Accuracy score: 0.9257244677858014
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9257244677858014

Tested category: toxic
Used algorithm: SVC
Accuracy score: 0.926209009346963
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.926209009346963

Tested category: severe_toxic
Used algorithm: SVC
Accuracy score: 0.9939666760448904
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9939666760448904

Tested category: obscene
Used algorithm: SVC
Accuracy score: 0.9569070618024946
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9569070618024946

Tested category: threat
Used algorithm: SVC
Accuracy score: 0.9964987964612836
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9964987964612836

Tested category: insult
Used algorithm: SVC
Accuracy score: 0.9536403138578886
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9536403138578886

Tested category

In [17]:
rf_clf0 = RandomForestClassifier(random_state=random_state)
rf_clfs = [RandomForestClassifier(random_state=random_state) for f in features]

In [18]:
rf_for_both_stages_classifier, rf_for_both_stages_results = \
    test_two_stage_classifier_with_algorithms(clf0=rf_clf0, clfs=rf_clfs)

Test data - first stage classification
Used algorithm: RandomForestClassifier
Accuracy score: 0.918847103691894
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.918847103691894

Tested category: toxic
Used algorithm: RandomForestClassifier
Accuracy score: 0.919003407421301
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.919003407421301

Tested category: severe_toxic
Used algorithm: RandomForestClassifier
Accuracy score: 0.9930913751602113
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9930913751602113

Tested category: obscene
Used algorithm: RandomForestClassifier
Accuracy score: 0.9513895401544281
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9513895401544281

Tested category: threat
Used algorithm: RandomForestClassifier
Accuracy score: 0.9963424927318766
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9963424927318766

Tested category: insult
Used algorithm: RandomForestClassifier
Accuracy score: 0.94

In [19]:
mnb_clf0 = MultinomialNB()
mnb_clfs = [MultinomialNB() for f in features]

In [20]:
mnb_for_both_stages_classifier, mnb_for_both_stages_results = \
    test_two_stage_classifier_with_algorithms(clf0=mnb_clf0, clfs=mnb_clfs)

Test data - first stage classification
Used algorithm: MultinomialNB
Accuracy score: 0.9321641814373691
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9321641814373691

Tested category: toxic
Used algorithm: MultinomialNB
Accuracy score: 0.9334927631373284
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9334927631373284

Tested category: severe_toxic
Used algorithm: MultinomialNB
Accuracy score: 0.9942949138766451
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9942949138766451

Tested category: obscene
Used algorithm: MultinomialNB
Accuracy score: 0.9595173340835912
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9595173340835912

Tested category: threat
Used algorithm: MultinomialNB
Accuracy score: 0.9967019913095126
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9967019913095126

Tested category: insult
Used algorithm: MultinomialNB
Accuracy score: 0.9545156147425677
F1 score: <function f1_score at 0x7

In [21]:
lr_clf0 = LogisticRegression(solver='saga', random_state=random_state)
lr_clfs = [LogisticRegression(solver='saga', random_state=random_state) for f in features]

In [22]:
lr_for_both_stages_classifier, lr_for_both_stages_classifier = \
    test_two_stage_classifier_with_algorithms(clf0=lr_clf0, clfs=lr_clfs)

Test data - first stage classification
Used algorithm: LogisticRegression
Accuracy score: 0.9258807715152083
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9258807715152083

Tested category: toxic
Used algorithm: LogisticRegression
Accuracy score: 0.9262715308387258
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9262715308387258

Tested category: severe_toxic
Used algorithm: LogisticRegression
Accuracy score: 0.9936228078401951
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9936228078401951

Tested category: obscene
Used algorithm: LogisticRegression
Accuracy score: 0.9588295976742005
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9588295976742005

Tested category: threat
Used algorithm: LogisticRegression
Accuracy score: 0.995592234830723
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.995592234830723

Tested category: insult
Used algorithm: LogisticRegression
Accuracy score: 0.9516865172403014
F1 scor

In [23]:
mnb_clf0_ih = MultinomialNB()
mnb_clfs_ih = [MultinomialNB() for f in features]
mnb_clfs_ih[-1] = LogisticRegression(solver='saga', random_state=random_state)  # LR for identity_hate

In [24]:
mnb_for_both_stages_except_ih_classifier, mnb_for_both_stages_except_ih_results = \
    test_two_stage_classifier_with_algorithms(clf0=mnb_clf0_ih, clfs=mnb_clfs_ih)

Test data - first stage classification
Used algorithm: MultinomialNB
Accuracy score: 0.9321641814373691
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9321641814373691

Tested category: toxic
Used algorithm: MultinomialNB
Accuracy score: 0.9334927631373284
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9334927631373284

Tested category: severe_toxic
Used algorithm: MultinomialNB
Accuracy score: 0.9942949138766451
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9942949138766451

Tested category: obscene
Used algorithm: MultinomialNB
Accuracy score: 0.9595173340835912
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9595173340835912

Tested category: threat
Used algorithm: MultinomialNB
Accuracy score: 0.9967019913095126
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9967019913095126

Tested category: insult
Used algorithm: MultinomialNB
Accuracy score: 0.9545156147425677
F1 score: <function f1_score at 0x7

In [25]:
mnb_svc_clf0 = MultinomialNB()
mnb_svc_clfs = [SVC(random_state=random_state) for f in features]

In [26]:
mnb_svc_classifier, mnb_svc_results = \
    test_two_stage_classifier_with_algorithms(clf0=mnb_svc_clf0, clfs=mnb_svc_clfs)

Test data - first stage classification
Used algorithm: MultinomialNB
Accuracy score: 0.9321641814373691
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9321641814373691

Tested category: toxic
Used algorithm: SVC
Accuracy score: 0.9334927631373284
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9334927631373284

Tested category: severe_toxic
Used algorithm: SVC
Accuracy score: 0.9939666760448904
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9939666760448904

Tested category: obscene
Used algorithm: SVC
Accuracy score: 0.9622057582293914
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9622057582293914

Tested category: threat
Used algorithm: SVC
Accuracy score: 0.9967176216824534
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9967176216824534

Tested category: insult
Used algorithm: SVC
Accuracy score: 0.9571571477695457
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9571571477695457

Tes

In [27]:
mnb_rf_clf0 = MultinomialNB()
mnb_rf_clfs = [RandomForestClassifier(random_state=random_state) for f in features]

In [28]:
mnb_rf_classifier, mnb_rf_results = \
    test_two_stage_classifier_with_algorithms(clf0=mnb_rf_clf0, clfs=mnb_rf_clfs)

Test data - first stage classification
Used algorithm: MultinomialNB
Accuracy score: 0.9321641814373691
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9321641814373691

Tested category: toxic
Used algorithm: RandomForestClassifier
Accuracy score: 0.9334615023914471
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9334615023914471

Tested category: severe_toxic
Used algorithm: RandomForestClassifier
Accuracy score: 0.9931851573978555
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9931851573978555

Tested category: obscene
Used algorithm: RandomForestClassifier
Accuracy score: 0.961033480258839
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.961033480258839

Tested category: threat
Used algorithm: RandomForestClassifier
Accuracy score: 0.9965925786989278
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9965925786989278

Tested category: insult
Used algorithm: RandomForestClassifier
Accuracy score: 0.956234955

In [29]:
mnb_lr_clf0 = MultinomialNB()
mnb_lr_clfs = [LogisticRegression(solver='saga', random_state=random_state) for f in features]

In [30]:
mnb_lr_classifier, mnb_lr_results = \
    test_two_stage_classifier_with_algorithms(clf0=mnb_lr_clf0, clfs=mnb_lr_clfs)

Test data - first stage classification
Used algorithm: MultinomialNB
Accuracy score: 0.9321641814373691
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9321641814373691

Tested category: toxic
Used algorithm: LogisticRegression
Accuracy score: 0.9334927631373284
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9334927631373284

Tested category: severe_toxic
Used algorithm: LogisticRegression
Accuracy score: 0.9936228078401951
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9936228078401951

Tested category: obscene
Used algorithm: LogisticRegression
Accuracy score: 0.9622682797211541
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9622682797211541

Tested category: threat
Used algorithm: LogisticRegression
Accuracy score: 0.9956547563224859
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9956547563224859

Tested category: insult
Used algorithm: LogisticRegression
Accuracy score: 0.9562036950201632
F1 score: 

In [31]:
svc_mnb_clf0 = SVC(random_state=random_state)
svc_mnb_clfs = [MultinomialNB() for f in features]

In [32]:
svc_mnb_classifier, svc_mnb_results = \
    test_two_stage_classifier_with_algorithms(clf0=svc_mnb_clf0, clfs=svc_mnb_clfs)

Test data - first stage classification
Used algorithm: SVC
Accuracy score: 0.9257244677858014
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9257244677858014

Tested category: toxic
Used algorithm: MultinomialNB
Accuracy score: 0.926209009346963
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.926209009346963

Tested category: severe_toxic
Used algorithm: MultinomialNB
Accuracy score: 0.9942949138766451
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9942949138766451

Tested category: obscene
Used algorithm: MultinomialNB
Accuracy score: 0.9563756291225108
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9563756291225108

Tested category: threat
Used algorithm: MultinomialNB
Accuracy score: 0.9967019913095126
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9967019913095126

Tested category: insult
Used algorithm: MultinomialNB
Accuracy score: 0.9521397980555816
F1 score: <function f1_score at 0x7ff3c993a378>

In [33]:
rf_mnb_clf0 = RandomForestClassifier(random_state=random_state)
rf_mnb_clfs = [MultinomialNB() for f in features]

In [34]:
rf_mnb_classifier, rf_mnb_results = \
    test_two_stage_classifier_with_algorithms(clf0=rf_mnb_clf0, clfs=rf_mnb_clfs)

Test data - first stage classification
Used algorithm: RandomForestClassifier
Accuracy score: 0.918847103691894
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.918847103691894

Tested category: toxic
Used algorithm: MultinomialNB
Accuracy score: 0.9190190377942418
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9190190377942418

Tested category: severe_toxic
Used algorithm: MultinomialNB
Accuracy score: 0.9942949138766451
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9942949138766451

Tested category: obscene
Used algorithm: MultinomialNB
Accuracy score: 0.9548438525743225
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9548438525743225

Tested category: threat
Used algorithm: MultinomialNB
Accuracy score: 0.9967019913095126
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9967019913095126

Tested category: insult
Used algorithm: MultinomialNB
Accuracy score: 0.950717434117978
F1 score: <function f1_score 

In [35]:
lr_mnb_clf0 = LogisticRegression(solver='saga', random_state=random_state)
lr_mnb_clfs = [MultinomialNB() for f in features]

In [36]:
lr_mnb_classifier, lr_mnb_results = \
    test_two_stage_classifier_with_algorithms(clf0=lr_mnb_clf0, clfs=lr_mnb_clfs)

Test data - first stage classification
Used algorithm: LogisticRegression
Accuracy score: 0.9258807715152083
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9258807715152083

Tested category: toxic
Used algorithm: MultinomialNB
Accuracy score: 0.9262715308387258
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9262715308387258

Tested category: severe_toxic
Used algorithm: MultinomialNB
Accuracy score: 0.9942949138766451
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9942949138766451

Tested category: obscene
Used algorithm: MultinomialNB
Accuracy score: 0.9560317609178155
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9560317609178155

Tested category: threat
Used algorithm: MultinomialNB
Accuracy score: 0.9967019913095126
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9967019913095126

Tested category: insult
Used algorithm: MultinomialNB
Accuracy score: 0.9513739097814874
F1 score: <function f1_score a

In [37]:
mnb_clf0 = MultinomialNB()
mnb_clfs = [MultinomialNB() for f in features]

In [38]:
mnb_for_both_stages_classifier, mnb_for_both_stages_results = \
    test_two_stage_classifier_with_algorithms(clf0=mnb_clf0, clfs=mnb_clfs)

Test data - first stage classification
Used algorithm: MultinomialNB
Accuracy score: 0.9321641814373691
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9321641814373691

Tested category: toxic
Used algorithm: MultinomialNB
Accuracy score: 0.9334927631373284
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9334927631373284

Tested category: severe_toxic
Used algorithm: MultinomialNB
Accuracy score: 0.9942949138766451
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9942949138766451

Tested category: obscene
Used algorithm: MultinomialNB
Accuracy score: 0.9595173340835912
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9595173340835912

Tested category: threat
Used algorithm: MultinomialNB
Accuracy score: 0.9967019913095126
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9967019913095126

Tested category: insult
Used algorithm: MultinomialNB
Accuracy score: 0.9545156147425677
F1 score: <function f1_score at 0x7

In [39]:
mnb_001_clf0 = MultinomialNB(alpha=0.01)
mnb_001_clfs = [MultinomialNB(alpha=0.01) for f in features]

In [40]:
mnb_001_classifier, mnb_001_results = \
    test_two_stage_classifier_with_algorithms(clf0=mnb_001_clf0, clfs=mnb_001_clfs)

Test data - first stage classification
Used algorithm: MultinomialNB
Accuracy score: 0.9320703991997249
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9320703991997249

Tested category: toxic
Used algorithm: MultinomialNB
Accuracy score: 0.9333989808996843
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9333989808996843

Tested category: severe_toxic
Used algorithm: MultinomialNB
Accuracy score: 0.994091719028416
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.994091719028416

Tested category: obscene
Used algorithm: MultinomialNB
Accuracy score: 0.9595642252024134
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9595642252024134

Tested category: threat
Used algorithm: MultinomialNB
Accuracy score: 0.9967332520553941
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9967332520553941

Tested category: insult
Used algorithm: MultinomialNB
Accuracy score: 0.9545468754884492
F1 score: <function f1_score at 0x7ff

In [41]:
bnb_clf0 = BernoulliNB()
bnb_clfs = [BernoulliNB() for f in features]

In [42]:
bnb_for_both_stages_classifier, bnb_for_both_stages_results = \
    test_two_stage_classifier_with_algorithms(clf0=bnb_clf0, clfs=bnb_clfs)

Test data - first stage classification
Used algorithm: BernoulliNB
Accuracy score: 0.6818124980462034
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.6818124980462034

Tested category: toxic
Used algorithm: BernoulliNB
Accuracy score: 0.6816718246897371
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.6816718246897371

Tested category: severe_toxic
Used algorithm: BernoulliNB
Accuracy score: 0.9046390946887993
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9046390946887993

Tested category: obscene
Used algorithm: BernoulliNB
Accuracy score: 0.7147769545781363
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.7147769545781363

Tested category: threat
Used algorithm: BernoulliNB
Accuracy score: 0.9958423207977742
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9958423207977742

Tested category: insult
Used algorithm: BernoulliNB
Accuracy score: 0.6993810372315483
F1 score: <function f1_score at 0x7ff3c993a378>

In [43]:
bnb_001_clf0 = BernoulliNB(alpha=0.01)
bnb_001_clfs = [BernoulliNB(alpha=0.01) for f in features]

In [44]:
bnb_001_classifier, bnb_001_results = \
    test_two_stage_classifier_with_algorithms(clf0=bnb_001_clf0, clfs=bnb_001_clfs)

Test data - first stage classification
Used algorithm: BernoulliNB
Accuracy score: 0.6816093031979743
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.6816093031979743

Tested category: toxic
Used algorithm: BernoulliNB
Accuracy score: 0.6815311513332708
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.6815311513332708

Tested category: severe_toxic
Used algorithm: BernoulliNB
Accuracy score: 0.8768951827190596
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.8768951827190596

Tested category: obscene
Used algorithm: BernoulliNB
Accuracy score: 0.7173715964862921
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.7173715964862921

Tested category: threat
Used algorithm: BernoulliNB
Accuracy score: 0.9928569195661009
F1 score: <function f1_score at 0x7ff3c993a378>
Recall score: 0.9928569195661009

Tested category: insult
Used algorithm: BernoulliNB
Accuracy score: 0.7013973553408984
F1 score: <function f1_score at 0x7ff3c993a378>